In [130]:
import sqlite3
import pandas as pd

# Creates DB if it does not exist.
connection = sqlite3.connect("smallCompany.db") # SQL: CREATE DATABASE smallCompany;

# Once connected, creates Cursor object to execute SQL commands.
cursor = connection.cursor()

# SQLite is a bit different from MySQL. Must execute 'PRAGMA foreign_keys = ON' every time when
# connecting to SQLite DB - otherwise foreign key constraint is ignored (can then insert foreign
# keys that do not exist; we do not want that).
sql = "PRAGMA foreign_keys = ON"
cursor.execute(sql)
# Commits changse to DB.
connection.commit()

## Creates 4 tables and these relations (through foreign keys)
![](DBschema.png "DB schema")

In [131]:
sql = """CREATE TABLE Employees (
            id int,
            last_name varchar(255),
            first_name varchar(255),
            start_date date,
            end_date date,
            number_children int,
            salary float,
            department_id int,
            PRIMARY KEY (id),
            FOREIGN KEY (department_id) REFERENCES Departments(id)
)"""
cursor.execute(sql)

# Commits/saves changse to DB.
connection.commit()

In [132]:
sql = """CREATE TABLE Departments (
            id int,
            department_name varchar(255),
            PRIMARY KEY (id)
)"""
cursor.execute(sql)
connection.commit()

In [133]:
sql = """CREATE TABLE Involvements (
            employee_id int,
            project_id int,
            FOREIGN KEY (employee_id) REFERENCES Employees(id),
            FOREIGN KEY (project_id) REFERENCES Projects(id)
)"""
cursor.execute(sql)
connection.commit()

In [134]:
sql = """CREATE TABLE Projects (
            id int,
            project_name varchar(255),
            start_date date,
            end_date date,
            PRIMARY KEY (id)
)"""
cursor.execute(sql)
connection.commit()

## Inserts values in 4 tables

In [135]:
sql = """INSERT INTO Departments (department_name, id)
         VALUES ("General Management", 10)
"""
cursor.execute(sql)

sql = """INSERT INTO Departments (department_name, id)
         VALUES ("Marketing", 20)
"""
cursor.execute(sql)

sql = """INSERT INTO Departments (department_name, id)
         VALUES ("Finance", 30)
"""
cursor.execute(sql)

sql = """INSERT INTO Departments (department_name, id)
         VALUES ("Sales", 40)
"""
cursor.execute(sql)

connection.commit()

In [136]:
sql = """INSERT INTO Employees (id, last_name, first_name, start_date, end_date, number_children, salary, department_id)
         VALUES (1, "last_name1", "first_name1", "2021-08-15", null, 2, 150000, 10)
"""
cursor.execute(sql)

sql = """INSERT INTO Employees (id, last_name, first_name, start_date, end_date, number_children, salary, department_id)
         VALUES (2, "last_name3", "first_name2", "2020-08-15", null, 1, 85000, 20)
"""
cursor.execute(sql)

sql = """INSERT INTO Employees (id, last_name, first_name, start_date, end_date, number_children, salary, department_id)
         VALUES (3, "last_name3", "first_name3", "2019-08-15", null, 0, 70000, 30)
"""
cursor.execute(sql)

sql = """INSERT INTO Employees (id, last_name, first_name, start_date, end_date, number_children, salary, department_id)
         VALUES (4, "last_name4", "first_name4", "2018-08-15", null, 4, 55000, 40)
"""
cursor.execute(sql)

connection.commit()

In [137]:
sql = """INSERT INTO Projects (id, project_name, start_date, end_date)
         VALUES (1, "project_name1", "2021-08-15", null)
"""
cursor.execute(sql)

sql = """INSERT INTO Projects (id, project_name, start_date, end_date)
         VALUES (2, "project_name2", "2022-01-15", null)
"""
cursor.execute(sql)

connection.commit()

In [138]:
sql = """INSERT INTO Involvements (employee_id, project_id)
         VALUES (1, 1)
"""
cursor.execute(sql)

sql = """INSERT INTO Involvements (employee_id, project_id)
         VALUES (4, 1)
"""
cursor.execute(sql)

connection.commit()

## Views DB tables/data

### Basic select

In [139]:
sql = "SELECT * FROM Employees"
table = pd.read_sql_query(sql, connection)
table

,id,last_name,first_name,start_date,end_date,number_children,salary,department_id
0,1,last_name1,first_name1,2021-08-15,None,2,150000.0,10
1,2,last_name3,first_name2,2020-08-15,None,1,85000.0,20
2,3,last_name3,first_name3,2019-08-15,None,0,70000.0,30
3,4,last_name4,first_name4,2018-08-15,None,4,55000.0,40


In [140]:
sql = "SELECT * FROM Departments"
table = pd.read_sql_query(sql, connection)
table

,id,department_name
0,10,General Management
1,20,Marketing
2,30,Finance
3,40,Sales


In [141]:
sql = "SELECT * FROM Involvements"
table = pd.read_sql_query(sql, connection)
table

,employee_id,project_id
0,1,1
1,4,1


In [142]:
sql = "SELECT * FROM Projects"
table = pd.read_sql_query(sql, connection)
table

,id,project_name,start_date,end_date
0,1,project_name1,2021-08-15,None
1,2,project_name2,2022-01-15,None


### Select

In [143]:
sql = """
SELECT last_name, first_name, number_children
FROM Employees
WHERE number_children >= 1
"""
table = pd.read_sql_query(sql, connection)
table

,last_name,first_name,number_children
0,last_name1,first_name1,2
1,last_name3,first_name2,1
2,last_name4,first_name4,4


In [144]:
sql = """
SELECT *
FROM Employees
WHERE number_children >= 1 AND first_name LIKE "f%"
"""
# "f%": if first_name starts with 'f'
table = pd.read_sql_query(sql, connection)
table

,id,last_name,first_name,start_date,end_date,number_children,salary,department_id
0,1,last_name1,first_name1,2021-08-15,None,2,150000.0,10
1,2,last_name3,first_name2,2020-08-15,None,1,85000.0,20
2,4,last_name4,first_name4,2018-08-15,None,4,55000.0,40


In [145]:
sql = """
SELECT *
FROM Employees
WHERE (number_children >= 1 OR first_name LIKE "f%") AND start_date >= "2020-01-01"
"""
table = pd.read_sql_query(sql, connection)
table

,id,last_name,first_name,start_date,end_date,number_children,salary,department_id
0,1,last_name1,first_name1,2021-08-15,None,2,150000.0,10
1,2,last_name3,first_name2,2020-08-15,None,1,85000.0,20


In [146]:
sql = """
SELECT *
FROM Employees
WHERE number_children <> 1
"""
# <> means !=
table = pd.read_sql_query(sql, connection)
table

,id,last_name,first_name,start_date,end_date,number_children,salary,department_id
0,1,last_name1,first_name1,2021-08-15,None,2,150000.0,10
1,3,last_name3,first_name3,2019-08-15,None,0,70000.0,30
2,4,last_name4,first_name4,2018-08-15,None,4,55000.0,40


In [147]:
sql = """
SELECT first_name AS first, last_name AS last
FROM Employees
WHERE last_name = "last_name3" AND first_name = "first_name3"
"""
table = pd.read_sql_query(sql, connection)
table

,first,last
0,first_name3,last_name3


In [148]:
sql = """
SELECT Employees.first_name, Employees.last_name, Departments.department_name
FROM Employees, Departments
"""
table = pd.read_sql_query(sql, connection)
table

,first_name,last_name,department_name
0,first_name1,last_name1,General Management
1,first_name1,last_name1,Marketing
2,first_name1,last_name1,Finance
3,first_name1,last_name1,Sales
4,first_name2,last_name3,General Management
5,first_name2,last_name3,Marketing
6,first_name2,last_name3,Finance
7,first_name2,last_name3,Sales
8,first_name3,last_name3,General Management
9,first_name3,last_name3,Marketing


In [149]:
# This is same as above.
sql = """
SELECT emp.first_name, emp.last_name, dep.department_name
FROM Employees AS emp, Departments AS dep
"""
table = pd.read_sql_query(sql, connection)
table

,first_name,last_name,department_name
0,first_name1,last_name1,General Management
1,first_name1,last_name1,Marketing
2,first_name1,last_name1,Finance
3,first_name1,last_name1,Sales
4,first_name2,last_name3,General Management
5,first_name2,last_name3,Marketing
6,first_name2,last_name3,Finance
7,first_name2,last_name3,Sales
8,first_name3,last_name3,General Management
9,first_name3,last_name3,Marketing


In [150]:
sql = """
SELECT SUM(emp.number_children) AS sum_children
FROM Employees AS emp
WHERE emp.end_date IS NULL
"""
table = pd.read_sql_query(sql, connection)
table

,sum_children
0,7


In [151]:
sql = """
SELECT AVG(emp.salary) AS average_salary
FROM Employees AS emp
WHERE emp.end_date IS NULL
"""
table = pd.read_sql_query(sql, connection)
table

,average_salary
0,90000.0


In [152]:
sql = """
SELECT COUNT(dep.ID) AS number_departments
FROM Departments AS dep
"""
table = pd.read_sql_query(sql, connection)
table

,number_departments
0,4


In [153]:
sql = """
SELECT proj.project_name AS project, COUNT(*) AS number_involved
FROM Projects AS proj, Involvements AS inv
WHERE inv.project_id = proj.ID
GROUP BY proj.ID
"""
table = pd.read_sql_query(sql, connection)
table

,project,number_involved
0,project_name1,2


In [154]:
# WHERE emp.department_id = dep.id 'joins' / combines rows from two tables.
sql = """
SELECT emp.first_name AS first, emp.last_name AS last, dep.department_name AS department
FROM Employees AS emp, Departments AS dep
WHERE emp.department_id = dep.id
"""
table = pd.read_sql_query(sql, connection)
table

,first,last,department
0,first_name1,last_name1,General Management
1,first_name2,last_name3,Marketing
2,first_name3,last_name3,Finance
3,first_name4,last_name4,Sales


In [155]:
# WHERE ... 'joins' / combines rows from two tables.
sql = """
SELECT emp.first_name AS first, emp.last_name AS last, dep.department_name AS department
FROM Employees AS emp, Departments AS dep
WHERE emp.department_id = dep.id AND dep.department_name = "Marketing"
"""
table = pd.read_sql_query(sql, connection)
table

,first,last,department
0,first_name2,last_name3,Marketing


In [156]:
# WHERE ... 'joins' / combines rows from three tables.
sql = """
SELECT proj.project_name AS project, emp.last_name AS last, emp.first_name AS first
FROM Employees AS emp, Involvements AS inv, Projects AS proj
WHERE inv.project_id = proj.id AND inv.employee_id = emp.id
"""
table = pd.read_sql_query(sql, connection)
table

,project,last,first
0,project_name1,last_name1,first_name1
1,project_name1,last_name4,first_name4


In [157]:
# Independent subquery.
sql = """
SELECT emp.first_name AS first, emp.last_name AS last, emp.salary
FROM Employees AS emp
WHERE emp.salary >= (
    SELECT AVG(em.salary)
    FROM Employees AS em
)
"""
table = pd.read_sql_query(sql, connection)
table

,first,last,salary
0,first_name1,last_name1,150000.0


In [158]:
sql = """
SELECT *
FROM Employees AS emp
ORDER BY emp.last_name ASC
"""
table = pd.read_sql_query(sql, connection)
table

,id,last_name,first_name,start_date,end_date,number_children,salary,department_id
0,1,last_name1,first_name1,2021-08-15,None,2,150000.0,10
1,2,last_name3,first_name2,2020-08-15,None,1,85000.0,20
2,3,last_name3,first_name3,2019-08-15,None,0,70000.0,30
3,4,last_name4,first_name4,2018-08-15,None,4,55000.0,40


In [159]:
# Multiple ordering/sorting.
sql = """
SELECT proj.project_name AS project, emp.last_name AS last, emp.first_name AS first
FROM Employees AS emp, Involvements AS inv, Projects AS proj
WHERE inv.project_id = proj.id AND inv.employee_id = emp.id
ORDER BY proj.project_name, emp.last_name ASC
"""
table = pd.read_sql_query(sql, connection)
table

,project,last,first
0,project_name1,last_name1,first_name1
1,project_name1,last_name4,first_name4


In [160]:
# Gives rows with index 0, 1, and 2.
sql = """
SELECT emp.last_name AS last, emp.first_name AS first, emp.salary
FROM Employees AS emp
ORDER BY emp.salary DESC
LIMIT 0, 3
"""
table = pd.read_sql_query(sql, connection)
table

,last,first,salary
0,last_name1,first_name1,150000.0
1,last_name3,first_name2,85000.0
2,last_name3,first_name3,70000.0


## Drops/deletes (all) tables

In [161]:
sql = "DROP TABLE IF EXISTS Involvements"
cursor.execute(sql)

sql = "DROP TABLE IF EXISTS Projects"
cursor.execute(sql)

sql = "DROP TABLE IF EXISTS Employees"
cursor.execute(sql)

sql = "DROP TABLE IF EXISTS Departments"
cursor.execute(sql)

connection.commit()

## Closes DB connection

In [162]:
# Closes DB connection.
connection.close()